### This notebook is used to generate a plot like figure_2 in the RHO-Loss Paper.

Definition of all needed variables and imports:

In [1]:
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import pickle
import os
import pprint
pp = pprint.PrettyPrinter(depth=3)
import numpy as np
from statistics import mean

cifar10_results_x = []
cifar10_results_y = []
cifar100_results_x = []
cifar100_results_y = []
cinic10_results_x = []
cinic10_results_y = []

Get baselines:

In [2]:
baseline = {
    'CIFAR10' : {},
    'CINIC10' : {},
    'CIFAR100': {}
}

baseline_path = '/home/alsch/Desktop/ML Reproducibillity Chellange/figure_2/0_baseline/'

for file in os.listdir(baseline_path):
    if file.endswith('.pickle'):
        with open(os.path.join(baseline_path, file), 'rb') as f:
            results = pickle.load(f)
            for i in range(len(results['seed'])):
                seed = results['seed'][i]
                best = max(results['result'][i]['val_acc'][:100])
                epoch = results['result'][i]['val_acc'][:100].index(best)
                baseline[file.split(sep='_')[0].upper()][seed] = (best, epoch)


pp.pprint(baseline)

{'CIFAR10': {42: (78.45999908447266, 96),
             47: (78.69000244140625, 97),
             74: (78.19000244140625, 87)},
 'CIFAR100': {42: (43.83000183105469, 88),
              47: (44.380001068115234, 70),
              74: (43.52000045776367, 94)},
 'CINIC10': {42: (72.38555908203125, 63),
             47: (71.87333679199219, 65),
             74: (72.06999969482422, 52)}}


Get speedups for big model

In [3]:
big_model_path = '/home/alsch/Desktop/ML Reproducibillity Chellange/figure_2/1_big_model/target'

big_model = {
    'CIFAR10' : [],
    'CINIC10' : [],
    'CIFAR100': []
}

for file in os.listdir(big_model_path):
    if file.endswith('.pickle'):
        seed = int(file.split('_')[-1].split('.')[0])
        dataset = file.split('_')[1]
        with open(os.path.join(big_model_path, file), 'rb') as f:
            results = pickle.load(f)
            acc = np.array(results['accuracy'])
            speedup = baseline[dataset][seed][1] / np.argmax(acc > baseline[dataset][seed][0])
            speedup = round(speedup, 2)
            big_model[dataset].append(speedup)

y = 4 - (1/3)
for dataset in big_model.keys():
    globals()[dataset.lower()+'_results_x'].append(round(mean(big_model[dataset]), 2))
    globals()[dataset.lower()+'_results_y'].append(y)
    y += 1/3

46.02000045776367
80.20999908447266
75.83777618408203
73.6500015258789
45.95000076293945
80.55999755859375
73.92555236816406
45.220001220703125
79.83000183105469
[4.6]
[3.59]
[2.98]


Get speedups for small model

In [4]:
small_model_path = '/home/alsch/Desktop/ML Reproducibillity Chellange/figure_2/2_small_model/target'

small_model = {
    'CIFAR10' : [],
    'CINIC10' : [],
    'CIFAR100': []
}

for file in os.listdir(small_model_path):
    if file.endswith('.pickle'):
        seed = int(file.split('_')[-1].split('.')[0])
        dataset = file.split('_')[1]
        with open(os.path.join(small_model_path, file), 'rb') as f:
            results = pickle.load(f)
            acc = np.array(results['accuracy'])
            speedup = baseline[dataset][seed][1] / np.argmax(acc > baseline[dataset][seed][0])
            if speedup != np.inf:
                speedup = round(speedup, 2)
                small_model[dataset].append(speedup)

y = 3 - (1/3)
for dataset in small_model.keys():
    print(dataset)
    globals()[dataset.lower()+'_results_x'].append(round(mean(small_model[dataset]), 2))
    globals()[dataset.lower()+'_results_y'].append(y)
    y += 1/3

CIFAR10
[4.6, 2.44]
CINIC10
[3.59, 2.05]
CIFAR100
[2.98, 1.3]


/tmp/ipykernel_25920/1106134241.py:16: RuntimeWarning: divide by zero encountered in long_scalars
  speedup = baseline[dataset][seed][1] / np.argmax(acc > baseline[dataset][seed][0])


Get speedups for no holdout data

In [5]:
no_holdout_path = '/home/alsch/Desktop/ML Reproducibillity Chellange/figure_2/3_no_holdout/target'

no_holdout = {
    'CIFAR10' : [],
    'CINIC10' : [],
    'CIFAR100': []
}

for file in os.listdir(no_holdout_path):
    if file.endswith('.pickle'):
        seed = int(file.split('_')[-1].split('.')[0])
        dataset = file.split('_')[1]
        with open(os.path.join(no_holdout_path, file), 'rb') as f:
            results = pickle.load(f)
            acc = np.array(results['accuracy'])
            speedup = baseline[dataset][seed][1] / np.argmax(acc > baseline[dataset][seed][0])
            if speedup != np.inf:
                speedup = round(speedup, 2)
                no_holdout[dataset].append(speedup)

print(no_holdout)
y = 2 - (1/3)
for dataset in no_holdout.keys():
    globals()[dataset.lower()+'_results_x'].append(round(mean(no_holdout[dataset]), 2))
    globals()[dataset.lower()+'_results_y'].append(y)
    y += 1/3

{'CIFAR10': [2.07], 'CINIC10': [1.7, 2.1], 'CIFAR100': [1.76, 2.14]}
CIFAR10
[4.6, 2.44, 2.07]
CINIC10
[3.59, 2.05, 1.9]
CIFAR100
[2.98, 1.3, 1.95]


/tmp/ipykernel_25920/21492539.py:16: RuntimeWarning: divide by zero encountered in long_scalars
  speedup = baseline[dataset][seed][1] / np.argmax(acc > baseline[dataset][seed][0])


Get speedups for architecture transfer

In [6]:
path_results = '/home/alsch/Desktop/ML Reproducibillity Chellange/figure_2/4_architecture/temp'
baseline_folder = 'baseline'
target_folder = 'target'

count = 0

arch_results = {
    'CIFAR10' : {},
    'CIFAR100': {},
    'CINIC10': {}
}

for file in os.listdir(os.path.join(path_results, baseline_folder)):
    if file.endswith('.pickle'):
        file_splitted = file.split('_')

        dataset = file_splitted[1]
        model = file_splitted[2]
        seed = int(file_splitted[-1].split('.')[0])

        file_splitted[0] = 'target'

        target_file = '_'.join(file_splitted)
        baseline_path = os.path.join(path_results, baseline_folder, file)
        target_path = os.path.join(path_results, target_folder, target_file)

        with open(baseline_path, 'rb') as f:
            results = pickle.load(f)
            base_acc = np.array(results['accuracy'])[:100]
        with open(target_path, 'rb') as f:
            results = pickle.load(f)
            target_acc = np.array(results['accuracy'])

        if model in arch_results[dataset].keys():
            arch_results[dataset][model].append(np.argmax(base_acc) / np.argmax(target_acc > np.max(base_acc)))
        else:
            arch_results[dataset][model] = []
            arch_results[dataset][model].append(np.argmax(base_acc) / np.argmax(target_acc > np.max(base_acc)))

y = 1 - (1/3)
for dataset in arch_results.keys():
    for model in arch_results[dataset]:
        globals()[dataset.lower()+'_results_x'].append(round(mean(arch_results[dataset][model]), 2))
        globals()[dataset.lower()+'_results_y'].append(y)
    y += 1/3

/tmp/ipykernel_25920/232864216.py:38: RuntimeWarning: divide by zero encountered in long_scalars
  arch_results[dataset][model].append(np.argmax(base_acc) / np.argmax(target_acc > np.max(base_acc)))
/tmp/ipykernel_25920/232864216.py:35: RuntimeWarning: divide by zero encountered in long_scalars
  arch_results[dataset][model].append(np.argmax(base_acc) / np.argmax(target_acc > np.max(base_acc)))


Get speedups for hyperparameter

In [7]:
hyperparameter = '/home/alsch/Desktop/ML Reproducibillity Chellange/figure_2/5_hyperparameters/target'

y_value = {
    'CIFAR10' : -(1/3),
    'CINIC10' : 0,
    'CIFAR100': +(1/3)
}


for folder in os.listdir(hyperparameter):
    for file in os.listdir(os.path.join(hyperparameter, folder)):
        if file.endswith('.pickle'):
            dataset = file.split('_')[1]
            seed = int(file.split('_')[3])
            with open(os.path.join(hyperparameter, folder, file), 'rb') as f:
                results = pickle.load(f)
                acc = np.array(results['accuracy'])
                speedup = baseline[dataset][seed][1] / np.argmax(acc > baseline[dataset][seed][0])
                speedup = round(speedup, 2)
                globals()[dataset.lower()+'_results_x'].append(speedup if speedup != np.inf else 0)
                globals()[dataset.lower()+'_results_y'].append(y_value[dataset])


/tmp/ipykernel_25920/3005694039.py:18: RuntimeWarning: divide by zero encountered in long_scalars
  speedup = baseline[dataset][seed][1] / np.argmax(acc > baseline[dataset][seed][0])


Show the plot

In [24]:
fig, ax = plt.subplots(figsize=(20, 5))

x = np.arange(0, 7.5, 0.1)

for i in range(5):
    if i % 2 == 0:
        ax.fill_between(x, -0.5+i, 0.5+i, color='whitesmoke')

plt.axvline(x=1, color='k', linestyle='dashed', linewidth=1)

ax.scatter(cifar10_results_x, cifar10_results_y, label='CIFAR10', color='blue', alpha=0.5, edgecolors='None', s=150)
ax.scatter(cifar100_results_x, cifar100_results_y, label='CIFAR100', color='orange', alpha=0.5, edgecolors='None', s=150)
ax.scatter(cinic10_results_x, cinic10_results_y, label='CINIC10', color='green', alpha=0.5, edgecolors='None', s=150)

ax.legend(fontsize=16).set_title("Dataset", prop={'size': 18})

ax.set_title('Speedup using RHO-Loss Selection', fontsize=20)

ax.set_xlim((0, 6))
ax.set_ylim((-0.5, 4.5))

ax.set_yticks([0, 1, 2, 3, 4])
ax.set_yticklabels(["Hyperparameter transfer", "Architecture transfer", "No holdout set", "Small irreducible loss model", "Default"], fontsize=10)
ax.tick_params(axis='y', labelsize=14)
ax.tick_params(axis='x', labelsize=14)

ax.set_xticks([0, 1, 3, 6])
ax.set_xticklabels([0, 'No speedup', '3x', '6x'])

fig.savefig('figure_2.pdf', bbox_inches='tight')
fig.show()